In [1]:
import torch
import os
os.environ['NEURITE_BACKEND'] = 'pytorch'
os.environ['VXM_BACKEND'] = 'pytorch'
import voxelmorph as vxm
import matplotlib.pyplot as plt
import glob
import re
import csv
from tifffile import imwrite
import numpy as np
from PIL import Image
import fnmatch
from collections import defaultdict
import shutil

from my_utils_v2 import Utils_v2

Original Tissue Path

In [3]:
tissue_path='/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered'
tissue_name='GCA033TIB_TISSUE01'
mask_path=f'/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/{tissue_name}_RetentionMask.tif'
output_path='/fs5/p_masi/rudravg/MxIF_Vxm_Registered'

In [4]:
mask_path

'/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_RetentionMask.tif'

Getting all Tissues

In [5]:
all_tissues=[]
for file in os.listdir(tissue_path):
    if fnmatch.fnmatch(file, f'{tissue_name}_*'):
        all_tissues.append(os.path.join(tissue_path, file))
print(len(all_tissues))

78


Putting each DAPI and it's corresponding stains into an array

In [6]:
filtered_tissues = list(filter(lambda x: 'RetentionMask' not in x, all_tissues))
print(len(filtered_tissues))
# Group the strings by round number
tissues_by_round = defaultdict(list)
for tissue in filtered_tissues:
    round_number = int(re.search('ROUND_(\\d+)', tissue).group(1))
    tissues_by_round[round_number].append(tissue)

# Sort the dictionary items by key (round number)
sorted_items = sorted(tissues_by_round.items())

# Sort each group so that the string containing "DAPI_DAPI" comes first, and convert the sorted dictionary items to a list
grouped_tissues = [sorted(group, key=lambda x: f'{tissue_name}_DAPI' not in x) for _, group in sorted_items]


77


In [7]:
sum=0
for group in grouped_tissues:
    sum+=len(group)
sum

77

In [8]:
grouped_tissues

[['/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_00.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_WGA_CY5_75ms_ROUND_00.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_AF_CY2_75ms_ROUND_00.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_SNA_CY3_50ms_ROUND_00.tif'],
 ['/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_01.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY2_75ms_ROUND_01.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY3_50ms_ROUND_01.tif',
  '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TIS

Declaring the Round 0 DAPI and Registering the DAPI's and stains together

In [9]:
round0_dapi = grouped_tissues[0][0]
round0_dapi,height,width = Utils_v2.load_tissues_for_overlap(round0_dapi,mask_path)

In [10]:
model_path='/home-local/rudravg/test_DAPI/1024_Dataset_V2/trial2_epochs/epoch_113/epoch_113.pth'
model,device=Utils_v2.load_model(model_path) #Write this code

/home/local/VANDERBILT/rudravg/miniconda3/envs/dapi_reg/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Copy the paths related to Round0, since we are not applying any transformations on them

In [11]:
new_dir = os.path.join(output_path, tissue_name)
os.makedirs(new_dir, exist_ok=True)

for file_path in grouped_tissues[0]:
    file,height,width=Utils_v2.load_tissues_for_overlap(file_path,mask_path)
    file=file*255.
    file = file[:height, :width]
    Image.fromarray(np.uint8(file)).save(os.path.join(new_dir, os.path.basename(file_path)))

In [12]:
print(new_dir)

/fs5/p_masi/rudravg/MxIF_Vxm_Registered/GCA033TIB_TISSUE01


In [13]:
for group in grouped_tissues:
    print (group)

['/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_00.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_WGA_CY5_75ms_ROUND_00.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_AF_CY2_75ms_ROUND_00.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_SNA_CY3_50ms_ROUND_00.tif']
['/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_01.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY2_75ms_ROUND_01.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY3_50ms_ROUND_01.tif', '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Backgroun

Now comes the actual registration lesgo

In [14]:
model_path='/home-local/rudravg/test_DAPI/1024_Dataset_V2/trial2_epochs/epoch_113/epoch_113.pth'
model,device=Utils_v2.load_model(model_path) #Write this code
i=0
for group in grouped_tissues[1:]:
    stains=len(group)
    i+=1
    print(f"Registering Round {i}, which has {stains} number of stains")
    if stains==1:
        print("Bruh where are all the tissues")
    elif stains==2:
        roundi_dapi = group[0]
        stain1=group[1]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        registered_dapi,registered_stain1=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        Image.fromarray(registered_dapi).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(registered_stain1).save(os.path.join(new_dir,os.path.basename(group[1])))
    elif stains==3:
        roundi_dapi = group[0]
        stain1=group[1]
        stain2=group[2]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        stain2,_,_ = Utils_v2.load_tissues_for_overlap(stain2,mask_path)
        registered_dapi,registered_stain1,registered_stain2=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,stain2=stain2,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_stain2 = registered_stain2*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        registered_stain2 = registered_stain2[:height, :width]
    
        Image.fromarray(registered_dapi).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(registered_stain1).save(os.path.join(new_dir,os.path.basename(group[1])))
        Image.fromarray(registered_stain2).save(os.path.join(new_dir,os.path.basename(group[2])))
    elif stains==4:
        roundi_dapi = group[0]
        stain1=group[1]
        stain2=group[2]
        stain3=group[3]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        stain2,_,_ = Utils_v2.load_tissues_for_overlap(stain2,mask_path)
        stain3,_,_ = Utils_v2.load_tissues_for_overlap(stain3,mask_path)
        registered_dapi,registered_stain1,registered_stain2,registered_stain3=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,stain2=stain2,stain3=stain3,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_stain2 = registered_stain2*255.
        registered_stain3 = registered_stain3*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        registered_stain2 = registered_stain2[:height, :width]
        registered_stain3 = registered_stain3[:height, :width]
        Image.fromarray(np.uint8(registered_dapi)).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(np.uint8(registered_stain1)).save(os.path.join(new_dir,os.path.basename(group[1])))
        Image.fromarray(np.uint8(registered_stain2)).save(os.path.join(new_dir,os.path.basename(group[2])))
        Image.fromarray(np.uint8(registered_stain3)).save(os.path.join(new_dir,os.path.basename(group[3])))
    elif stains==5:
        roundi_dapi = group[0]
        stain1=group[1]
        stain2=group[2]
        stain3=group[3]
        stain4=group[4]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        stain2,_,_ = Utils_v2.load_tissues_for_overlap(stain2,mask_path)
        stain3,_,_ = Utils_v2.load_tissues_for_overlap(stain3,mask_path)
        stain4,_,_ = Utils_v2.load_tissues_for_overlap(stain4,mask_path)
        registered_dapi,registered_stain1,registered_stain2,registered_stain3,registered_stain4=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,stain2=stain2,stain3=stain3,stain4=stain4,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_stain2 = registered_stain2*255.
        registered_stain3 = registered_stain3*255.
        registered_stain4 = registered_stain4*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        registered_stain2 = registered_stain2[:height, :width]
        registered_stain3 = registered_stain3[:height, :width]
        registered_stain4 = registered_stain4[:height, :width]
        Image.fromarray(np.uint8(registered_dapi)).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(np.uint8(registered_stain1)).save(os.path.join(new_dir,os.path.basename(group[1])))
        Image.fromarray(np.uint8(registered_stain2)).save(os.path.join(new_dir,os.path.basename(group[2])))
        Image.fromarray(np.uint8(registered_stain3)).save(os.path.join(new_dir,os.path.basename(group[3])))
        Image.fromarray(np.uint8(registered_stain4)).save(os.path.join(new_dir,os.path.basename(group[4])))
    elif stains==6:
        roundi_dapi = group[0]
        stain1=group[1]
        stain2=group[2]
        stain3=group[3]
        stain4=group[4]
        stain5=group[5]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        stain2,_,_ = Utils_v2.load_tissues_for_overlap(stain2,mask_path)
        stain3,_,_ = Utils_v2.load_tissues_for_overlap(stain3,mask_path)
        stain4,_,_ = Utils_v2.load_tissues_for_overlap(stain4,mask_path)
        stain5,_,_ = Utils_v2.load_tissues_for_overlap(stain5,mask_path)
        registered_dapi,registered_stain1,registered_stain2,registered_stain3,registered_stain4,registered_stain5=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,stain2=stain2,stain3=stain3,stain4=stain4,stain5=stain5,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_stain2 = registered_stain2*255.
        registered_stain3 = registered_stain3*255.
        registered_stain4 = registered_stain4*255.
        registered_stain5 = registered_stain5*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        registered_stain2 = registered_stain2[:height, :width]
        registered_stain3 = registered_stain3[:height, :width]
        registered_stain4 = registered_stain4[:height, :width]
        registered_stain5 = registered_stain5[:height, :width]
        Image.fromarray(np.uint8(registered_dapi)).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(np.uint8(registered_stain1)).save(os.path.join(new_dir,os.path.basename(group[1])))
        Image.fromarray(np.uint8(registered_stain2)).save(os.path.join(new_dir,os.path.basename(group[2])))
        Image.fromarray(np.uint8(registered_stain3)).save(os.path.join(new_dir,os.path.basename(group[3])))
        Image.fromarray(np.uint8(registered_stain4)).save(os.path.join(new_dir,os.path.basename(group[4])))
        Image.fromarray(np.uint8(registered_stain5)).save(os.path.join(new_dir,os.path.basename(group[5])))
    elif stains==7:
        roundi_dapi = group[0]
        stain1=group[1]
        stain2=group[2]
        stain3=group[3]
        stain4=group[4]
        stain5=group[5]
        stain6=group[6]
        roundi_dapi,_,_ = Utils_v2.load_tissues_for_overlap(roundi_dapi,mask_path)
        stain1,_,_ = Utils_v2.load_tissues_for_overlap(stain1,mask_path)
        stain2,_,_ = Utils_v2.load_tissues_for_overlap(stain2,mask_path)
        stain3,_,_ = Utils_v2.load_tissues_for_overlap(stain3,mask_path)
        stain4,_,_ = Utils_v2.load_tissues_for_overlap(stain4,mask_path)
        stain5,_,_ = Utils_v2.load_tissues_for_overlap(stain5,mask_path)
        stain6,_,_ = Utils_v2.load_tissues_for_overlap(stain6,mask_path)
        registered_dapi,registered_stain1,registered_stain2,registered_stain3,registered_stain4,registered_stain5,registered_stain6=Utils_v2.register_multiple_tissues_with_overlap(moving=roundi_dapi,fixed=round0_dapi,
                                                                                          stain1=stain1,stain2=stain2,stain3=stain3,stain4=stain4,stain5=stain5,stain6=stain6,model=model,device=device)
        registered_dapi = registered_dapi*255.
        registered_stain1 = registered_stain1*255.
        registered_stain2 = registered_stain2*255.
        registered_stain3 = registered_stain3*255.
        registered_stain4 = registered_stain4*255.
        registered_stain5 = registered_stain5*255.
        registered_stain6 = registered_stain6*255.
        registered_dapi = registered_dapi[:height, :width]
        registered_stain1 = registered_stain1[:height, :width]
        registered_stain2 = registered_stain2[:height, :width]
        registered_stain3 = registered_stain3[:height, :width]
        registered_stain4 = registered_stain4[:height, :width]
        registered_stain5 = registered_stain5[:height, :width]
        registered_stain6 = registered_stain6[:height, :width]
        Image.fromarray(np.uint8(registered_dapi)).save(os.path.join(new_dir,os.path.basename(group[0])))
        Image.fromarray(np.uint8(registered_stain1)).save(os.path.join(new_dir,os.path.basename(group[1])))
        Image.fromarray(np.uint8(registered_stain2)).save(os.path.join(new_dir,os.path.basename(group[2])))
        Image.fromarray(np.uint8(registered_stain3)).save(os.path.join(new_dir,os.path.basename(group[3])))
        Image.fromarray(np.uint8(registered_stain4)).save(os.path.join(new_dir,os.path.basename(group[4])))
        Image.fromarray(np.uint8(registered_stain5)).save(os.path.join(new_dir,os.path.basename(group[5])))
        Image.fromarray(np.uint8(registered_stain6)).save(os.path.join(new_dir,os.path.basename(group[6])))

Registering Round 1, which has 5 number of stains
Registering Round 2, which has 4 number of stains
Registering Round 3, which has 4 number of stains
Registering Round 4, which has 4 number of stains
Registering Round 5, which has 4 number of stains
Registering Round 6, which has 4 number of stains
Registering Round 7, which has 4 number of stains
Registering Round 8, which has 4 number of stains
Registering Round 9, which has 4 number of stains
Registering Round 10, which has 4 number of stains
Registering Round 11, which has 4 number of stains
Registering Round 12, which has 4 number of stains
Registering Round 13, which has 4 number of stains
Registering Round 14, which has 4 number of stains
Registering Round 15, which has 4 number of stains
Registering Round 16, which has 4 number of stains
Registering Round 17, which has 4 number of stains
Registering Round 18, which has 4 number of stains


In [25]:
sox9_after='/fs5/p_masi/rudravg/MxIF_Vxm_Registered/GCA020TIB_TISSUE01/AF_Removed/ROUND_02_CY5_GCA020TIB_TISSUE01_CD45_normalized_corrected.tif'
sox9_before='/fs5/p_masi/rudravg/MxIF_Vxm_Registered/GCA020TIB_TISSUE01/Unregistered/AF_Removed/ROUND_02_CY5_GCA020TIB_TISSUE01_CD45_normalized_corrected.tif'
average_dapi='/home-local/rudravg/dapi_raw/Set02/GCA020TIB/AFRemoved/GCA020TIB_TISSUE01_DAPI.tif'
sox9_before=np.array(Image.open(sox9_before))
sox9_after=np.array(Image.open(sox9_after))
average_dapi=np.array(Image.open(average_dapi))

In [26]:
Utils_v2.calculate_ncc(sox9_before.ravel(),average_dapi.ravel())

array([0.14160487])

In [27]:
Utils_v2.calculate_ncc(sox9_after.ravel(),average_dapi.ravel())

array([0.14275191])